In [5]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

from flask import Flask, request, jsonify, render_template

In [19]:
app = Flask(__name__)

# Fetch data from Analytik table
def fetch_analytik_data(user_input):
    engine = create_engine("sqlite:///datenbank.db")
    query = f"""SELECT * FROM analytik
                WHERE name = '{user_input}';"""
    table_ana = pd.read_sql(query, engine)
    table_ana['name'] = ['']*len(table_ana)
    if table_ana.empty:
        return None
    return table_ana.to_dict(orient='records')

# Fetch data from transp. Metabolite table
def fetch_metabolite_data(user_input):
    engine = create_engine("sqlite:///datenbank.db")
    query = f"""SELECT * FROM transportierte_metabolite
                WHERE name = '{user_input}';"""
    table_meta = pd.read_sql(query, engine)
    table_meta['name'] = ['']*len(table_meta)
    if table_meta.empty:
        return None
    return table_meta.to_dict(orient='records')

# Define a Python function that takes user input and search user input in database
def search_database(user_input, categories):
    # SQL code
    engine = create_engine("sqlite:///datenbank.db")
    query = f"""SELECT * FROM namen
                WHERE name = '{user_input}';"""
    table_name = pd.read_sql(query, engine)
    name = table_name['name'][0]
    namen = table_name['weitere_namen'][0]
    abk = table_name['abkürzungen'][0]
    
    result = {'Name': {'name': name, 'namen': namen, 'abk': abk}}
    
    # Überprüfen, was bei den Checkboxen ausgewählt wurde
    # Biochemie
    if 'Biochemie' in categories:
        query = f"""SELECT * FROM biochemie
                    WHERE name = '{user_input}';"""
        table_bio = pd.read_sql(query, engine)
        mol_masse = table_bio['molekulare_masse'][0]
        aminos = table_bio['aminosäuren'][0]
        oligo = table_bio['oligomerisierung'][0]
        glyko = table_bio['glykolisierung'][0]
        bind = table_bio['bindungsmotiv'][0]
        enzym = table_bio['enzymfunktion'][0]
        biochemie = {'Molekulare Masse': mol_masse,
                    'Aminosäuren': aminos,
                    'Oligomerisierung': oligo,
                    'Glykolisierung': glyko,
                    'Bindungsmotiv': bind,
                    'Enzymfunktion': enzym}
        result['Biochemie'] = biochemie
    # Funktion
    if 'Funktion' in categories:
        query = f"""SELECT * FROM funktionen
                    WHERE name = '{user_input}';"""
        table_func = pd.read_sql(query, engine)
        synth_gewebe = table_func['synthetisierendes_gewebe'][0]
        elek = table_func['elektrophorese'][0]
        immun = table_func['immunsystem'][0]
        haupt = table_func['hauptfunktion'][0]
        funktion = {'Synthetisierendes Gewebe': synth_gewebe,
                   'Elektrophorese': elek,
                   'Immunsystem': immun,
                   'Hauptfunktion': haupt}
        result['Funktion'] = funktion
        # transp. Metabolite
        # Wenn Hauptfunktion des Parameters 'Transport' ist, werden Infos zu den Metaboliten angezeigt
        if 'Transport' in funktion['Hauptfunktion']:
            meta_data = fetch_metabolite_data(user_input)
            if meta_data:
                result['Transportierte Metabolite'] = meta_data
        
    # Diagnostik
    if 'Diagnostik' in categories:
        query = f"""SELECT * FROM diagnostik
                    WHERE name = '{user_input}';"""
        table_diag = pd.read_sql(query, engine)
        biomat = table_diag['biomaterial'][0]
        ref = table_diag['referenzbereich'][0]
        hohe_werte = table_diag['erhöhte_werte'][0]
        niedrige_werte = table_diag['erniedrigte_werte'][0]
        diag = {'Biomaterial': biomat,
               'Referenzbereich': ref,
               'Erhöhte Werte': hohe_werte,
               'Erniedrigte Werte': niedrige_werte}
        result['Diagnostik'] = diag
    # Analytik (mehrere Einträge möglich)
    if 'Analytik' in categories:
        analytik_data = fetch_analytik_data(user_input)
        if analytik_data:
            result['Analytik'] = analytik_data
    # Abreicherung (auch mehrere Einträge möglich)
    if 'Abreicherung' in categories:
        query = f"""SELECT * FROM abreicherung
                    WHERE name = '{user_input}';"""
        table_ab = pd.read_sql(query, engine)
        abr = []
        for i in range(len(table_ab)):
            methodenname = table_ab['methodenname'][i]
            methode = table_ab['methode'][i]
            abr.append({'Methodenname': methodenname,
                        'Methode': methode})
        result['Abreicherung'] = abr
        
    return result

# Define a route for the root URL
@app.route('/', methods=['GET'])
def index():
    return render_template('index.html') 

# Define the route for executing the function
@app.route('/execute-function', methods=['POST'])
def execute_function():
    data = request.get_json()
    user_input = data['input']
    categories = data.get('categories', [])
    categories.append('Name')
    result = search_database(user_input, categories)
    return jsonify(result)

@app.route('/get-suggestions', methods=['POST'])
def get_suggestions():
    data = request.get_json()
    user_input = data['input']
    engine = create_engine("sqlite:///datenbank.db")
    query = f"""SELECT DISTINCT name FROM namen
                WHERE name LIKE '%{user_input}%';"""
    suggestions = pd.read_sql(query, engine)['name'].tolist()
    
    return jsonify({'suggestions': suggestions})

@app.route('/save_edited_method', methods=['POST'])
def save_edited_method():
    data = request.get_json()
    text_title = data.get('textTitle', '')
    text_description = data.get('textDescription', '')
    user_input = data['input']
    old_title = data['oldMethodTitle']
    # update database
    conn = sqlite3.connect('datenbank.db')
    cursor = conn.cursor()
    query = """ UPDATE abreicherung
                SET methodenname = ?,
                    methode = ?
                WHERE name = ?
                AND methodenname = ?;"""
    values = (text_title, text_description, user_input, old_title)
    cursor.execute(query, values)
    conn.commit()
    
@app.route('/save_new_method', methods=['POST'])
def save_new_method():
    data = request.get_json()
    text_title = data.get('textTitle', '')
    text_description = data.get('textDescription', '')
    user_input = data['input']
    # update database
    conn = sqlite3.connect('datenbank.db')
    cursor = conn.cursor()
    query = "INSERT INTO abreicherung (name, methodenname, methode) VALUES (?, ?, ?)"
    values = (user_input, text_title, text_description)
    cursor.execute(query, values)
    conn.commit()
    
@app.route('/delete_method', methods=['POST'])
def delete_method():
    data = request.get_json()
    text_title = data.get('textTitle', '')
    user_input = data['input']
    # update database
    conn = sqlite3.connect('datenbank.db')
    cursor = conn.cursor()
    query = """ DELETE FROM abreicherung
                WHERE name = ?
                AND methodenname = ?"""
    values = (user_input, text_title)
    cursor.execute(query, values)
    conn.commit()
    
@app.route('/save_in_database', methods=['POST'])
def save_in_database():
    data = request.get_json()
    table = data.get('tableName', '')
    neue_einträge = data.get('newEntries', '')
    user_input= data['input']
    
    table_dict = {
        'allgemein-section': 'namen',
        'biochemie-section': 'biochemie',
        'funktion-section': 'funktionen',
        'diagnostik-section': 'diagnostik',
        'abreicherung-section': 'abreicherung'
    }
    
    # Tabelle identifizieren
    table_database = table_dict[table]
    # Datenbank updaten
    conn = sqlite3.connect('datenbank.db')
    cursor = conn.cursor()
    if table_database == 'namen':
        query = """ UPDATE namen
                    SET name = ?,
                        weitere_namen = ?,
                        abkürzungen = ?
                    WHERE name = ?;"""
        values = (neue_einträge[0], neue_einträge[1], neue_einträge[2], user_input)
        cursor.execute(query, values)
        conn.commit()
        # falls Name des Parameters geändert wurde, muss dieser in allen Tabellen geändert werden
        if neue_einträge[0] != user_input:
            for tab in list(table_dict.values()):
                query = f"""UPDATE {tab}
                            SET name = ?
                            WHERE name = ?;"""
                values = (neue_einträge[0], user_input)
                cursor.execute(query, values)
                conn.commit()
                print(tab, neue_einträge[0], user_input)
            # Parametername in analytik-Tabelle updaten
            query = """ UPDATE analytik
                        SET name = ?
                        WHERE name = ?;"""
            values = (neue_einträge[0], user_input)
            cursor.execute(query, values)
            conn.commit()
    elif table_database == 'biochemie':
        query = """ UPDATE biochemie
                    SET molekulare_masse = ?,
                        aminosäuren = ?,
                        oligomerisierung = ?,
                        glykolisierung = ?,
                        bindungsmotiv = ?,
                        enzymfunktion = ?
                    WHERE name = ?;"""
        values = (neue_einträge[0], neue_einträge[1], neue_einträge[2], neue_einträge[3],
                  neue_einträge[4], neue_einträge[5], user_input)
        cursor.execute(query, values)
        conn.commit()
    elif table_database == 'funktionen':
        query = """ UPDATE funktionen
                    SET synthetisierendes_gewebe = ?,
                        elektrophorese = ?,
                        immunsystem = ?,
                        hauptfunktion = ?,
                    WHERE name = ?;"""
        values = (neue_einträge[0], neue_einträge[1], neue_einträge[2], neue_einträge[3], user_input)
        cursor.execute(query, values)
        conn.commit()
    elif table_database == 'transportierte_metabolite':
        query = """ UPDATE transportierte_metabolite
                    SET metabolit_name = ?,
                        interaktion = ?,
                    WHERE name = ?;"""
        values = (neue_einträge[0], neue_einträge[1], user_input)
        cursor.execute(query, values)
        conn.commit()
    elif table_database == 'diagnostik':
        query = """ UPDATE diagnostik
                    SET biomaterial = ?,
                        referenzbereich = ?,
                        erhöhte_werte = ?,
                        erniedrigte_werte = ?,
                    WHERE name = ?;"""
        values = (neue_einträge[0], neue_einträge[1], neue_einträge[2], neue_einträge[3], user_input)
        cursor.execute(query, values)
        conn.commit()
    elif table_database == 'abreicherung':
        query = """ UPDATE abreicherung
                    SET labor = ?,
                        methodenname = ?,
                        methode = ?,
                    WHERE name = ?;"""
        values = (neue_einträge[0], neue_einträge[1], user_input)
        cursor.execute(query, values)
        conn.commit()
        
@app.route('/save_new_entry', methods=['POST'])
def save_new_entry():
    data = request.get_json()
    parameter = data.get('paraName', '')
    para_namen = data.get('paraNamen', '')
    weitere_namen = data.get('weitereNamen', '')
    abk = data.get('paraAbk', '')
    molmasse = data.get('molMasse', '')
    aminos = data.get('aminoAcids', '')
    oligo = data.get('oligomerisierung', '')
    glyko = data.get('glykolisierung', '')
    bind = data.get('bindMotiv', '')
    enzym = data.get('enzymFunktion', '')
    synth = data.get('synthGewebe', '')
    elek = data.get('elektrophorese', '')
    immun = data.get('immunsystem', '')
    haupt = data.get('hauptfunktion', '')
    meta = data.get('transMeta', '')
    inter = data.get('interaktion', '')
    biomat = data.get('biomaterial', '')
    ref = data.get('referenz', '')
    hoch = data.get('hoheWerte', '')
    niedrig = data.get('niedrigeWerte', '')
    meth_name = data.get('methodenName', '')
    metho = data.get('methode', '')
    
    query = """INSERT INTO namen (name, weitere_namen, abkürzungen) VALUES (?, ?, ?)"""

@app.route('/', methods=['GET'])
def update():
    # Run datenbank_update.py as a subprocess
    try:
        subprocess.run(["python", "datenbank_update.py"], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Update process failed with error code {e.returncode}")
    
    # Render the template
    return render_template('index.html') 

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.21.150:8000/ (Press CTRL+C to quit)
172.16.21.150 - - [20/Feb/2024 15:47:49] "GET / HTTP/1.1" 200 -
172.16.21.150 - - [20/Feb/2024 15:47:50] "GET /check-update-status HTTP/1.1" 404 -
172.16.21.150 - - [20/Feb/2024 15:47:51] "GET / HTTP/1.1" 200 -
172.16.21.150 - - [20/Feb/2024 15:47:51] "GET /static/css/style.css HTTP/1.1" 200 -
172.16.21.150 - - [20/Feb/2024 15:47:51] "GET /static/js/script.js HTTP/1.1" 200 -
172.16.21.150 - - [20/Feb/2024 15:47:51] "GET /static/images/dodo.png HTTP/1.1" 200 -
172.16.21.150 - - [20/Feb/2024 15:47:51] "GET /check-update-status HTTP/1.1" 404 -
172.16.21.150 - - [20/Feb/2024 15:47:51] "GET /favicon.ico HTTP/1.1" 404 -
[2024-02-20 15:47:57,869] ERROR in app: Exception on /save_new_entry [POST]
Traceback (most recent call last):
  File "C:\Users\l.nguyen quoc\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\l.nguyen quoc\An

anti-TPO aTPO


In [10]:
engine = create_engine("sqlite:///datenbank.db")
query = 'SELECT * FROM biochemie'
pd.read_sql(query, engine)

,name,molekulare_masse,aminosäuren,oligomerisierung,glykolisierung,bindungsmotiv,enzymfunktion
0,Transthyretin,55 kDa,127 aa (Monomer),Homotetramer,None,None,None
1,Fibrinogen3,340 kDa,Heterohexamer,Glykoprotein,None,None,None
